Stanislas Deneuville - Emmanuel Ferrandi - Pol Grisart - Marine Médard
# Project of data science :  Face recognition in a video and counting
12/11/2018

## Part I : Cleaning

In [1]:
import cv2
import sys
import os
import numpy as np

__face_detection__ : dectect the faces in a photograph.


In : 
* imagePath : name of the file containing the image in the format jpeg for example

Out : 
* face_locations : list of the location of all faces in the picture in parameter
* height : height of the input image
* width : width of the input image


In [2]:
def detectFaces(imagePath):
    cascPath = "haarcascade_frontalface_default.xml"

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height = image.shape[0]
    width = image.shape[1] 

    # Detect faces in the image
    face_locations = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(20, 20),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    return face_locations,height,width

__image_generation__ : generate a black image with white circle whoes location in gave in parameter


In : 
* face_locations : list containing the locations of the white rectangles that will be added to the picture
* height : height of the input image
* width : width of the input image

Out : 
* BWimage : list of list  representing a black picture with white rectangles


In [3]:
def toBlackAndWhite(face_locations,height,width):
    BWimage = np.zeros((height,width))
    height = BWimage.shape[0]
    
    for (x, y, w, h) in face_locations:
        for i in range(h):
            for j in range(w):
                BWimage[y+j,x+i]=255 #255 is black
                
    return BWimage

__resize_image__ : resize the image gave in parameter in a 50x50 pixels image 


In : 
* image : list of list  representing an image in white and black

Out : 
* face_locations : list of list representing the same image than in parameter but with the dimensions 50x50


In [4]:
def resize_image(image) :
    final_image = cv2.resize(image, (50,50), interpolation = cv2.INTER_AREA) 
    return final_image

__save_image__ : save the image in parameter in the good repository according to nb_circles


In : 
* image : image that we want to save
* nb_faces : int, the number of faces detected in the image
* cpt : int, the numero of the image that is being cleaned

Out : 
* int : 0 if everything is ok


In [5]:
def save_image(image, nb_faces, cpt) :
    path = "./train_set/"+str(nb_faces)+"/"
    if (not os.path.isdir(path)) :
        os.mkdir(path)
    cv2.imwrite(path+"image"+str(cpt)+".jpg",image)
    print("Image saved in : "+path+"image"+str(cpt)+".jpg")
    return 0

__cleaning__ : clean an image so that we can count faces in it thanks to an ANN

In : 
* image : name of the file containing the image in the format jpeg for example
* cpt : the numero of the image that is being cleaned


In [6]:
def cleaning(image, cpt) :
    face_locations,height,width = detectFaces(image)
    nb_faces = len(face_locations)
    
    newImage = toBlackAndWhite(face_locations,height,width)
    
    final_image = resize_image(newImage)
    
    save_image(final_image, nb_faces, cpt)

__cleaAll__ : clean all images in a directory

In : 
* name_dir : string, name of the directory where we want to clean all the images



In [ ]:
def cleanAll(name_dir) :
    cpt = 0
    liste_files = os.listdir(name_dir)
    for name_f in liste_files :
        if ('.jpg' in name_f) :
            cleaning(name_dir + '/' + name_f, cpt)
            cpt +=1
        
cleanAll('Group2a')    

Image saved in : ./train_set/3/image0.jpg
Image saved in : ./train_set/4/image1.jpg
Image saved in : ./train_set/5/image2.jpg
Image saved in : ./train_set/7/image3.jpg
Image saved in : ./train_set/4/image4.jpg
Image saved in : ./train_set/4/image5.jpg
Image saved in : ./train_set/4/image6.jpg
Image saved in : ./train_set/3/image7.jpg
Image saved in : ./train_set/4/image8.jpg
Image saved in : ./train_set/4/image9.jpg
Image saved in : ./train_set/2/image10.jpg
Image saved in : ./train_set/4/image11.jpg
Image saved in : ./train_set/3/image12.jpg
Image saved in : ./train_set/3/image13.jpg
Image saved in : ./train_set/7/image14.jpg
Image saved in : ./train_set/2/image15.jpg
Image saved in : ./train_set/3/image16.jpg
Image saved in : ./train_set/5/image17.jpg
Image saved in : ./train_set/5/image18.jpg
Image saved in : ./train_set/12/image19.jpg
Image saved in : ./train_set/6/image20.jpg
Image saved in : ./train_set/5/image21.jpg
Image saved in : ./train_set/4/image22.jpg
Image saved in : ./t

Image saved in : ./train_set/9/image189.jpg
Image saved in : ./train_set/5/image190.jpg
Image saved in : ./train_set/2/image191.jpg
Image saved in : ./train_set/3/image192.jpg
Image saved in : ./train_set/16/image193.jpg
Image saved in : ./train_set/5/image194.jpg
Image saved in : ./train_set/2/image195.jpg
Image saved in : ./train_set/4/image196.jpg
Image saved in : ./train_set/4/image197.jpg
Image saved in : ./train_set/5/image198.jpg
Image saved in : ./train_set/8/image199.jpg
Image saved in : ./train_set/5/image200.jpg
Image saved in : ./train_set/6/image201.jpg
Image saved in : ./train_set/5/image202.jpg
Image saved in : ./train_set/4/image203.jpg
Image saved in : ./train_set/2/image204.jpg
Image saved in : ./train_set/8/image205.jpg
Image saved in : ./train_set/2/image206.jpg
Image saved in : ./train_set/5/image207.jpg
Image saved in : ./train_set/5/image208.jpg
Image saved in : ./train_set/10/image209.jpg
Image saved in : ./train_set/3/image210.jpg
Image saved in : ./train_set/6

Image saved in : ./train_set/4/image375.jpg
Image saved in : ./train_set/2/image376.jpg
Image saved in : ./train_set/3/image377.jpg
Image saved in : ./train_set/3/image378.jpg
Image saved in : ./train_set/2/image379.jpg
Image saved in : ./train_set/2/image380.jpg
Image saved in : ./train_set/14/image381.jpg
Image saved in : ./train_set/5/image382.jpg
Image saved in : ./train_set/8/image383.jpg
Image saved in : ./train_set/3/image384.jpg
Image saved in : ./train_set/3/image385.jpg
Image saved in : ./train_set/3/image386.jpg
Image saved in : ./train_set/5/image387.jpg
Image saved in : ./train_set/8/image388.jpg
Image saved in : ./train_set/3/image389.jpg
Image saved in : ./train_set/4/image390.jpg
Image saved in : ./train_set/2/image391.jpg
Image saved in : ./train_set/3/image392.jpg
Image saved in : ./train_set/3/image393.jpg
Image saved in : ./train_set/6/image394.jpg
Image saved in : ./train_set/3/image395.jpg
Image saved in : ./train_set/7/image396.jpg
Image saved in : ./train_set/8/